In [1]:
#Author: Heeirthan Shanthan, Date of submission: 09/24/23, Homework 2

In [2]:
import pandas as pd
import numpy as np
import cpi
from datetime import date

C:\Users\heeir\AppData\Local\Programs\Python\Python311\Lib\site-packages\cpi\__init__.py:42: StaleDataWarning: CPI data is out of date. To accurately inflate to today's dollars, you must run `cpi.update()`.
  warnings.warn(StaleDataWarning())


In [3]:
df = pd.read_csv('/home/shanthah-erau.edu/Downloads/county_info.csv')

FileNotFoundError: [Errno 2] No such file or directory: '/home/shanthah-erau.edu/Downloads/county_info.csv'

In [ ]:
df.head()

In [ ]:
#making sure this column only has year 2019
df.cost_of_living_yr.unique()

In [ ]:
#Step 1

In [ ]:
#compute annual growth rate of population and append this column to dataset as it becomes new feature
start_pop = df['pop_in_2000']
end_pop = df['pop_in_later_year']
total_pop_growth = ((end_pop-start_pop)/start_pop)*100
annual_pop_growth = total_pop_growth/(df['pop_ref_later_year']-2000)
print(annual_pop_growth)
df['annual_pop_gr'] = annual_pop_growth
df.to_csv('county_info.csv')
df.head()

In [ ]:
#Step 2

In [ ]:
#compute annual house value and append this column to dataset as it becomes new feature
start_houseval= df['median_house_value_2000']
end_houseval = df['median_house_value_2017']
total_houseval_growth = ((end_houseval- start_houseval)/start_houseval)*100
annual_houseval_growth = total_houseval_growth/(2017-2000)
print(annual_houseval_growth)
df['annual_house_gr'] = annual_houseval_growth
df.to_csv('county_info.csv')
df.head()

In [ ]:
#Step 3

In [ ]:
#annual birth rate, assumed average of rate of the two intervals was the goal for this step.
birth_rateint1 = (df['birth_per_1000_int1']/(df['births_to_yr_int1']-df['births_from_yr_int1']))
birth_rateint2 = (df['birth_per_1000_int2']/(df['births_to_yr_int2']-df['births_from_yr_int2']))
avg_birthrate = (birth_rateint1+birth_rateint2)/2 
print(birth_rateint1)
df['annual_birth_gr'] = avg_birthrate
df.to_csv('county_info.csv')
df.head()

In [ ]:
#Step 4

In [ ]:
#adjust all dollar figures to 2022/03/01 using python CPI package
df['median_house_income_2017'] = df['median_house_income_2017'].apply(lambda row: cpi.inflate(row,date(2017,1,1),to=date(2022,3,1)))
df['cost_of_living_usd'] = df['cost_of_living_usd'].apply(lambda row: cpi.inflate(row,date(2019,1,1),to=date(2022,3,1)))
df[['state', 'county', 'median_house_income_2017', 'cost_of_living_usd']]

In [ ]:
#set features, target, drop rows lacking entries for our features
features = ['pop_foreign_born', 'adult_obes_rate', 'pop_per_sq_mi', 'pop_percent_urban', 'poverty_pct', 
            'median_house_income_2017', 'cost_of_living_usd', 'annual_pop_gr', 'annual_house_gr']
target = ['median_house_value_2000']
df = df[features+target]
df = df.dropna()

In [ ]:
#Step 5

In [ ]:
#split data into training, va;idation, test sets
#40% test, 60% training data split
#50% split between test and validation
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test=train_test_split(df[features],df[target],test_size = 0.4, random_state = 0)
x_val,x_test,y_val,y_test=train_test_split(x_test,y_test,test_size = 0.5, random_state = 0)

In [ ]:
y_test

In [ ]:
from sklearn.linear_model import LinearRegression
lr = LinearRegression()

In [ ]:
#train LR model and check validation score
lr.fit(x_train, y_train)
lr.score(x_val,y_val)

In [ ]:
#Step 6

In [ ]:
#what does obesity rate have to do with median house value in 2000?
#why does value decrease so much if annual hosue value growth rate increases by 1
#might be due to correlation...
for _ in zip(features, lr.coef_[0]):
    print(_[0],_[1])

In [ ]:
#Step 7

In [ ]:
df.corr()


In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
numeric_df = df.select_dtypes(include='number')

fig,ax = plt.subplots()
ax = sns.heatmap(df.corr(),vmin=-1,vmax=1,cmap="vlag",annot = True)
ax.set_xticklabels(ax.get_xticklabels(),rotation=90)
ax.set_yticklabels(ax.get_yticklabels(),rotation=0)
ax.set_title('median house value 2000 heat map');
#we can see the correlation as median house value decreases as obesity increases, but not as correlated as I thought
#strongest positive correlation between median house value and cost of living and house income 2017.
#also strong negative correlation between poverty and house income 2017

In [ ]:
features


In [ ]:
from itertools import combinations

In [ ]:
coef_dct = dict(zip(features,[[],[],[],[],[],[],[],[],[]]))

In [ ]:
#training for each featue subset as done in lectures previously
for i in range(1,len(features)+1):
    for feature_subset in combinations(features,i):
        feature_subset = list(feature_subset)
        df_tmp = df[feature_subset+target]
        df_tmp = df_tmp.dropna()
        x_train,x_test,y_train,y_test = train_test_split(df_tmp[feature_subset],
                                                         df_tmp[target],
                                                         test_size = 0.4,
                                                         random_state = 0)
        x_val,x_test,y_val,y_test = train_test_split(x_test,
                                                     y_test,
                                                     test_size = 0.5,
                                                     random_state = 0)
        lr = LinearRegression()
        lr.fit(x_train,y_train)
        lr.score(x_val,y_val)
        for item in zip(feature_subset, lr.coef_[0]):
            coef_dct[item[0]].append(item[1])

In [ ]:
df_coef = pd.DataFrame(coef_dct)

In [ ]:
from sklearn.decomposition import PCA

In [ ]:
pca = PCA(n_components=9)
pca.fit(x_train)

In [ ]:
#our first PC looks to good to be true,with 98% of variance being captured
#I dont think one linear combination can describe almost all variability
print(pca.explained_variance_ratio_)

In [ ]:
#std is extremely high, obviously not normalized...
df.describe()

In [ ]:
#Step 8
# let's normalize the data first
from sklearn import preprocessing

In [ ]:
scaler = preprocessing.StandardScaler()

In [ ]:
#Fitting scaler to training data
scaler.fit(x_train)

In [ ]:
x_train.head()

In [ ]:
#now, we look at the statistics again, std way better
pd.DataFrame(scaler.transform(x_train)).describe()

In [ ]:
#we can perform PCA again on our scaled data:
# fits the PCA to the scaled data
x_train_scaled = scaler.transform(x_train)
pca = PCA()
pca.fit(x_train_scaled)

In [ ]:
#this is more like it, shows that several components are needed to describe variance
print(pca.explained_variance_ratio_)

In [ ]:
#  see how the coefficients of the PCs vary in linear models

pca_features = ['pc'+str(i) for i in range(1,10)]
df_pca = pd.DataFrame(pca.transform(x_train_scaled),columns = pca_features)
coef_dct = dict(zip(pca_features,[[],[],[],[],[],[],[],[],[]]))

In [ ]:
# trained on all subsets of PCs.
for i in range(1,len(pca_features)+1):
    for feature_subset in combinations(pca_features,i):
        feature_subset = list(feature_subset)
        df_tmp = df_pca[feature_subset]
        df_tmp = df_tmp.dropna()
        lr = LinearRegression()
        lr.fit(df_tmp,y_train)
        lr.score(pd.DataFrame(pca.transform(scaler.transform(x_val)),columns=pca_features)[feature_subset],
                 y_val)
        for item in zip(feature_subset, lr.coef_[0]):
            coef_dct[item[0]].append(item[1])

In [ ]:
#coefficients for each feature combination is the same, good.
pd.DataFrame(coef_dct)

In [ ]:
#how the features are weighted in the first PC(how important are they in defining the first PC)
pca.components_[0] 

In [ ]:
#Step 9

In [ ]:
first_pc_coefficients = pca.components_[0]

# display the linear combination of features for the first principal component
#shows how much they contribute to PC1 
for feature, coefficient in zip(features, first_pc_coefficients):
    print(f"{feature}: {coefficient:.4f}")


In [ ]:
#Step 10:

In [ ]:
num_components = 9  # number of principal components 
pca = PCA(n_components=num_components)
x_train_pcs = pca.fit_transform(x_train_scaled)  # x_train_pcs will contain the principal components
validation_scores = []  # i use this array to store the r^2 scores for each component
x_val_scaled = scaler.transform(x_val)  # scaling validation set just as for training data

# transforming the validation data into PC dimensions using pca.tranform
x_val_pcs = pca.transform(x_val_scaled)  # now, validation data x_val_pcs is represented by PC components

for k in range(1,num_components+1):
    # training the lr model using the first k principal components
    model = LinearRegression()
    model.fit(x_train_pcs[:, :k], y_train)  # training all rows and first k components for subset
    
    # Evaluate the model on the validation set
    score = model.score(x_val_pcs[:, :k], y_val)  # selecting all rows and first k components for subset
    
    validation_scores.append(score)  # storing the R^2 scores in the array


In [ ]:
#Step 11 & 12

In [ ]:
#plotting cumulative explained variance and validation scores vs number of components
plt.plot(range(0, num_components), validation_scores, marker='o', linestyle='-')
plt.plot(range(0, num_components), np.cumsum(pca.explained_variance_ratio_), marker='o', linestyle='-')
plt.xlabel('Number of Components')
plt.ylabel('R^2 Score/VE')
plt.title('R^2 Score vs. Number of Components')
plt.legend(['R^2 Score', 'VER'])
plt.show()

In [ ]:
#interesting how validation scores decrease from PC6 to PC7. 
#might be due to overfitting as we add less meaningful variance?
validation_scores

In [ ]:
#Citations: https://www.w3schools.com/python/python_lambda.asp
# https://vitalflux.com/pca-explained-variance-concept-python-example/
#previous lectures
#python help documentation